## Ⅰ. Intro

### 1. 테이블의 내용
1. 고객이 각 귀속년도별
1. 근로 / 사업 / 기타 소득에 대한 예상환급액과 수수료를 조회한 뒤
1. 결제를 하거나 하지 않은 여부를 저장

### 2. 컬럼 정보
- age: 고객의 만 나이입니다. 
- gender: 고객의 성별입니다.
- year: 소득이 발생한 연도(귀속년도)입니다.
- refund: 예상환급액입니다.
- fee: 수수료입니다.
- has_paid: 수수료를 결제했는지의 여부
- income_근로: 고객의 근로소득(월급/일용직급여)
- income_사업: 고객의 사업소득(프리랜서 소득)
- income_기타: 고객의 기타소득(그외 기타 소득)

### 3. 질문

- 고객의 결제여부에 영향을 미치는 요인들은 무엇인가요? 
- 고객의 수수료 결제금액의 합을 높히기 위해서는 어떻게 해야 할까요?

### 4. 개념파악 
- 납부세액 혹은 환급세액의 결정방법
- 소득을 3범주로 하여, 기장의무가 달라진다.  
https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=fobi52&logNo=221270380307

### 5. EDA

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [2]:
df=pd.read_csv('Jobis_3o3.csv')

FileNotFoundError: [Errno 2] File b'Jobis_3o3.csv' does not exist: b'Jobis_3o3.csv'

In [ ]:
df.info()

In [ ]:
df.describe()

#### 1) 지불여부를 숫자로 바꾸고 수수료를 곱한 새로운 변수(Target)를 생성한다.
#### 2) 환급액에서 수수료를 뺀 gain이라는 변수를 생성하고, profit의 비율을 변수 pr로 생성한다.

In [ ]:
df['has_paid']=df['has_paid'].replace(True,1).replace(False,0)

In [ ]:
df['Target']=df.has_paid*df.fee

#### 3) 환급액과 수수료를 고려한 순 환급액과 이익률 변수를 생성

In [ ]:
pr=(df.refund-df.fee)/df.refund
df['pr']=pr
gain=df.refund-df.fee
df['gain']=gain

#### 4) 나이를 카테고리화

In [ ]:
cat_1=pd.cut(df.age,bins=[-1,21,24,26,29,34,37,48,100]).astype('str').str.rstrip(']').str.lstrip('(')
df['ag_cat']=cat_1

In [ ]:
df['근로yn']=df.income_근로.notnull()
df['사업yn']=df.income_사업.notnull()
df['기타yn']=df.income_기타.notnull()
df['소득notnull여부']=pd.concat([df.income_근로.notnull(),df.income_사업.notnull(),df.income_기타.notnull()],axis=1).sum(1).astype('str')

#### 5) 성별컬럼 결측값처리

In [ ]:
df.loc[df.gender=='-',:]

In [ ]:
df.query("has_paid==1").shape[0]

- 성별 값이 '-'인 데이터는 22건이 존재한다. 그리고 소득원에 대한 정보가 없다. (위 3가지 컬럼에 집계되지 않는 소득이 있는 것으로 판단한다.) 
- 이중 실제 결제 된 데이터가 4건에 해당한다. 이는, 총 10만건중 결제데이터 6만4643건의 데이터규모에서 극히 일부분이며, 성별 추정의  
  근거가   적기때문에 삭제한다.

In [ ]:
df=df.loc[df.gender!='-',:]

In [ ]:
df.query("refund <0 and has_paid ==1.0")

- 환급세액이 -인 경우는 기납부 세액이 결정세액보다 적은 경우이므로 세액의 특성상 존재하는 데이터이다.  
- 이 데이터들은 전체 데이터에서 수수료중 빈도수가 높은 400과 4300과 8500에 해당하는 데이터로 삭제하지 않는다.

In [ ]:
df.fee.value_counts()

- 수수료는 범주형 변수인 것으로 판단된다.

#### 6) 상관관계 파악

In [ ]:
corr=df.corr()

In [ ]:
plt.figure(figsize=(15, 15));
sns.heatmap(corr,
            vmax=0.8,
            linewidths=0.01,
            square=True,
            annot=True,
            cmap='YlGnBu');
plt.title('Feature Correlation')

- 상관계수 히트맵으로는 결제여부와 상관관계 있는 변수는 없는 것으로 판단된다.
- 다만, 결제금액인 Target을 보면, 상식적으로 생각하는 것처럼 환급액과 수수료에 약한 상관관계가 있다.
- 추가로, 사업소득과 결제금액은 뚜렷한 양의 상관관계가 있으며, 기타소득과도 결제금액은 어느정도의 상관관계가 있다.

####  7) 범주형 변수간의 상관관계 파악
#### - Cramer's V
#####   범주형 변수간 상관관계 파악
#####   비교 대상 범주 대상이 3개 이상
https://www.kaggle.com/chrisbss1/cramer-s-v-correlation-matrix

In [ ]:
df

In [ ]:
data = df[[i for i in df.columns if i in ('gender','ag_cat','has_paid','근로yn','사업yn','기타yn','소득notnull여부','year')]]

data.head()

In [ ]:
from sklearn import preprocessing

label = preprocessing.LabelEncoder()
data_encoded = pd.DataFrame() 

for i in data.columns :
  data_encoded[i]=label.fit_transform(data[i])

In [ ]:
data_encoded.head()

In [ ]:
from scipy.stats import chi2_contingency
import numpy as np


def cramers_V(var1,var2) :
  crosstab =np.array(pd.crosstab(var1,var2, rownames=None, colnames=None)) # Cross table building
  stat = chi2_contingency(crosstab)[0] # Keeping of the test statistic of the Chi2 test
  obs = np.sum(crosstab) # Number of observations
  mini = min(crosstab.shape)-1 # Take the minimum value between the columns and the rows of the cross table
  return (stat/(obs*mini))

In [ ]:
rows= []

for var1 in data_encoded:
  col = []
  for var2 in data_encoded :
    cramers =cramers_V(data_encoded[var1], data_encoded[var2]) # Cramer's V test
    col.append(round(cramers,2)) # Keeping of the rounded value of the Cramer's V  
  rows.append(col)
  
cramers_results = np.array(rows)
df1 = pd.DataFrame(cramers_results, columns = data_encoded.columns, index =data_encoded.columns)



df1

In [ ]:
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.figure(figsize=(15, 15));
sns.heatmap(df1,
            vmax=0.8,
            linewidths=0.01,
            square=True,
            annot=True,
            cmap='YlGnBu');
plt.title('Feature Correlation')

#### 통계적으로 변수간의 선형 상관관계는 없는 것으로 판단된다.

***

## Ⅱ. 고객의 결제여부에 영향을 미치는 요인 분석

### 1. 머신러닝 분류 모델을 통해 지불여부에 대해 데이터를 학습시키고 feature importance를 출력한 후 분석을 진행
#### 1) XGB분류모델을 통해 지불여부를 target value로 설정하고, 나머지 변수를 학습

In [ ]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

In [ ]:
X=df.reindex(['gender','year', 'refund', 'fee','income_근로','income_사업', 'income_기타','gain','age'],axis=1)
X=X.fillna(0)
y=df.has_paid
X=pd.get_dummies(X)
X.iloc[:,:8]=scaler.fit_transform(X.iloc[:,:8])
X

In [ ]:
y

In [ ]:
xgb_clf=XGBClassifier(n_estimators=100,random_state=42,learning_rate=0.3,use_label_encoder=False)
xgb_clf.fit(X,y)

#### 2) 변수중요도 파악

In [ ]:
fig, ax = plt.subplots()
plot_importance(xgb_clf,ax=ax)

- 피어슨상관계수 히트맵에서 결제금액과 (사업소득 기타소득)이 상관관계가 있었는데, 여기서도 feature importance기준으로 상위에 속한다.
- 제일 높은 변수중요도를 가진 변수는 사업소득이고,두번쨰가 나이,3번째가 근로소득, 네번째가 환급액,5번째가 기타 소득에 해당한다.  
    => <U>이는 삼쩜삼 서비스를 이용해서 결제하는 주요 고객이 프리랜서일 수 있다는 의미로 해석할 수 있다.</U>
- 변수에 각 소득원의 유무(예: 근로yn)를 넣고 학습시켰을 때 소득원의 유무는 변수중요도가 높지 않았다. 

### 2. income 사업 변수 및 소득 데이터 분석
    - 편의를 위해 두번째로 변수중요도가 높은 나이대 그룹 데이터로 나이대별 사업소득의 평균을 시각화한다.
    - 그래프에서 보는 것처럼 결제고객의 사업소득 금액대 평균이 미결제고객의 금액대 보다 모든 연령그룹에서 훨씬 낮은 특징이 있다.
    - 다만, 최빈값을 기준으로 보면 다른 인사이트를 얻을 수 있다.  
    - 20세 이전과 26세에서 29세그룹, 34세부터 고연령층까지 사업소득의 최빈값이 구매고객 쪽이 더 높다.
    - 이를 사용하면 연령별로 income 소득의 최빈값을 기준으로하여 결제율을 상승시킬 수 있다.

#### 1) 연령별 결제여부별 각 소득 파악

#### (1) 사업소득

In [ ]:
ag_p0=df.loc[df.income_사업.notnull(),:].query("has_paid==0").groupby('ag_cat')['income_사업'].median()
ag_p0.name='미결제고객'
ag_p0.plot()
ag_p1=df.loc[df.income_사업.notnull(),:].query("has_paid==1").groupby('ag_cat')['income_사업'].median()
ag_p1.name='결제고객'
ag_p1.plot()
plt.legend()

- 최빈값

In [ ]:
f1= lambda x: x.mode().min()
ag_p0=df.loc[df.income_사업.notnull(),:].query("has_paid==0").groupby('ag_cat')['income_사업'].agg(f1)
ag_p0.name='미결제고객'
ag_p0.plot()
ag_p1=df.loc[df.income_사업.notnull(),:].query("has_paid==1").groupby('ag_cat')['income_사업'].agg(f1)
ag_p1.name='결제고객'
ag_p1.plot()
plt.legend()

- 연령별 사업소득의 최빈값과 결제율 상승

In [ ]:
# 연령별 최빈값
ag_p1

In [ ]:
# 나이대별 결제율
l1=[]
l2=[]
for i in df.ag_cat.unique():
    str_expr="ag_cat == @i"
    str_expr1="ag_cat == @i and has_paid ==1"
    l1.append(df.query(str_expr1).shape[0]/df.query(str_expr).shape[0])
    l2.append(i)
    
age_cvr=pd.DataFrame([l2,l1]).T
age_cvr.columns=['age_bin','cvr']
age_cvr=age_cvr.set_index('age_bin').sort_values('cvr',ascending=False)
age_cvr

In [ ]:
# 사업 소득 변수에 최빈값 이상인 데이터의 나이대별 결제율
l1=[]
l2=[]
for i in np.arange(0,ag_p1.shape[0]):
        l1.append(ag_p1.index[i])
        l2.append(df.query("income_사업>= @i and ag_cat==@ag_p1.index[@i]").has_paid.value_counts()[1]/df.query("income_사업>= @i and ag_cat==@ag_p1.index[@i]").shape[0])

age_mode_cvr=pd.DataFrame([l1,l2]).T
age_mode_cvr.columns=['age_bin','cvr']    
age_buis_cvr=age_mode_cvr.set_index('age_bin').sort_values('cvr',ascending=False)
age_buis_cvr

In [ ]:
(age_buis_cvr-age_cvr).sort_values(by='cvr',ascending=False).mean()

- 연령별 사업소득 최빈값으로 기준을 설정해 결제율이 6% 증가했다.
- <U> xgboost 모델의 feature selection에서 나타낸 것처럼, income_사업 소득의 변화로 구매에 영향을 미칠 수 있음을 확인했다.</U>

#### (2) 기타소득

In [ ]:
f1= lambda x: x.mode()[0]
ag_p0=df.loc[df.income_기타.notnull(),:].query("has_paid==0").groupby('ag_cat')['income_기타'].agg(f1)
ag_p0.name='미결제고객'
ag_p0.plot()
ag_p1=df.loc[df.income_기타.notnull(),:].query("has_paid==1").groupby('ag_cat')['income_기타'].agg(f1)
ag_p1.name='결제고객'
ag_p1.plot()
plt.legend()

In [ ]:
# 기타 소득 변수에 최빈값 이상인 데이터의 나이대별 결제율
l1=[]
l2=[]
for i in np.arange(0,ag_p1.shape[0]):
        l1.append(ag_p1.index[i])
        l2.append(df.query("income_기타>= @i and ag_cat==@ag_p1.index[@i]").has_paid.value_counts()[1]/df.query("income_기타>= @i and ag_cat==@ag_p1.index[@i]").shape[0])

age_mode_cvr=pd.DataFrame([l1,l2]).T
age_mode_cvr.columns=['age_bin','cvr']    
age_etc_cvr=age_mode_cvr.set_index('age_bin').sort_values('cvr',ascending=False)
age_etc_cvr

In [ ]:
(age_etc_cvr-age_cvr).sort_values(by='cvr',ascending=False).mean()

- <U>나이별 최빈값이 결제고객이 높으므로 이를 기준으로한 데이터의 결제율은 전체보다 사업소득은 6.5%, 기타 소득은 2.35%의 향상이 있다.</U>

- 다만, 근로소득의 경우에는 최빈값으로 하면 전 연령대에 걸쳐 0이 최빈값이며 결제고객이 미결제고객보다 소득 수치가 낮다.

#### (3) 근로소득

In [ ]:
ag_p0=df.loc[df.income_근로.notnull(),:].query("has_paid==0").groupby('ag_cat')['income_근로'].agg(f1)
ag_p0.name='미결제고객'
ag_p0.plot()
ag_p1=df.loc[df.income_근로.notnull(),:].query("has_paid==1").groupby('ag_cat')['income_근로'].agg(f1)
ag_p1.name='결제고객'
ag_p1.plot()
plt.legend()

- 평균으로 기준을 설정하면 대체로 결제여부와 상관없이 비슷하며,34세 이후부터 급격한 성장을 보인다. 
- 그러므로 연령별 평균을 기준으로 하겠다.

In [ ]:
ag_p0=df.loc[df.income_근로.notnull(),:].query("has_paid==0").groupby('ag_cat')['income_근로'].mean()
ag_p0.name='미결제고객'
ag_p0.plot()
ag_p1=df.loc[df.income_근로.notnull(),:].query("has_paid==1").groupby('ag_cat')['income_근로'].mean()
ag_p1.name='결제고객'
ag_p1.plot()
plt.legend()

In [ ]:
# 기타 소득 변수에 최빈값 이상인 데이터의 나이대별 결제율
l1=[]
l2=[]
for i in np.arange(0,ag_p1.shape[0]):
        l1.append(ag_p1.index[i])
        l2.append(df.query("income_근로>= @i and ag_cat==@ag_p1.index[@i]").has_paid.value_counts()[1]/df.query("income_근로>= @i and ag_cat==@ag_p1.index[@i]").shape[0])

age_mode_cvr=pd.DataFrame([l1,l2]).T
age_mode_cvr.columns=['age_bin','cvr']    
age_lab_cvr=age_mode_cvr.set_index('age_bin').sort_values('cvr',ascending=False)
age_lab_cvr

- 그래프에서 보는 것처럼 20세 이하에는 거의 기준 설정전과 차이가 없다.
- 그러나, 34세 이후 기울기가 급격히 증가하는 것처럼, 결제율의 개선도 눈에 띄게 증가한다.
- 결제율의 개선수치의 평균도 xgboost의 featureimportance에서 3등인 것처럼, 사업소득보단 적지만 기타소득보다는 높다.

In [ ]:
(age_lab_cvr-age_cvr).sort_values(by='cvr',ascending=False).mean()

In [ ]:
(age_lab_cvr-age_cvr).sort_values(by='cvr',ascending=False)

### 3. AGE 변수 분석 

#### 1) 통계적 방법으로 독립성 검정을 통해 나이 카테고리변수와 결제여부컬럼의 독립성을 확인한다.

- 독립성 검정은  두 factor 변수사이에 '귀무가설 : 독립이다. / 대립가설 : 독립이 아니다.' 를 의미한다.
- 카이제곱검정을 통한 P_value가 0.05보다 작으면 귀무가설은 기각되고 대립가설이 채택된다.  
- 주어진 데이터에서 나이카테고리와 지불여부에 대한 독립성검정시 pvalue가 0.05보다 작아 대립가설이 채택되고 서로 관련이 있다고 볼 수 있다.

In [ ]:
from scipy.stats import chi2_contingency
df_age=pd.crosstab(df.ag_cat,df.has_paid)
p_value=chi2_contingency(df_age)[1]
(p_value,p_value<0.05)

In [ ]:
# 나이대별 결제 데이터 합의 규모를 보기 위한 것이다.
a=df.groupby('ag_cat')['has_paid'].sum().sort_values(ascending=False)
a

- 21-24세, 26-29세가 가장 결제 규모가 높고 34세까지는 높은 편에 속한다.
- age_cvr은 위에서 만든 나이별 전체 데이터의 결제율데이터이다.
- 20세 이하의 데이터가 1등이기 때문에, 결제 규모의 순위와는 차이가 있다고 볼 수 있다.
- 하지만, 34세 이상의 나이가 하위권이고 그 이전의 나이대는 상위권인것처럼 전체적인 추세는 비슷하다.
- <U>결국 34세 이하의 연령에 해당하는지의 여부가 결제여부에 영향을 미치는 것으로 볼 수 있다.</U>

In [ ]:
sns.barplot(x=a.index,y=a)

In [ ]:
sns.barplot(x=age_cvr.index,y=age_cvr.cvr)

### 4. refund 변수 와 year 변수 분석 
#### 1) 히스토그램과 value_counts를 통한 refund 변수 분석
    - 결제여부에 따라 환불액의 종류는 미결제 데이터가 1745개 결제데이터가 1521개로 차이가 있다.
    - 그리고 상위빈도 환불액의 경우, 크기차이가 있어 빈도수와 환불액을 곱한 수치의 합을 비교하겠다.
    - 이렇게 하면 높은 환불액에 빈도수가 높을 경우 더 높은 수치를 얻을 수 있다. 

In [ ]:
plt.hist(df.query("has_paid==0")['refund'],bins=40)

In [ ]:
plt.hist(df.query("has_paid==1")['refund'],bins=40)


In [ ]:
df.query("has_paid==0")['refund'].describe()

In [ ]:
df.query("has_paid==0")['refund'].value_counts()

In [ ]:
df.query("has_paid==0")['refund'].value_counts()/df.shape[0]

In [ ]:
df_r0=df.query("has_paid==0")['refund'].value_counts()
df_r0=df_r0.reset_index()
df_r00=(df_r0[['index']].transpose()*df_r0.refund).T

In [ ]:
df_r1=df.query("has_paid==1")['refund'].value_counts()
df_r1=df_r1.reset_index()
df_r11=(df_r1[['index']].transpose()*df_r1.refund).T

In [ ]:
df_r11.sum()>df_r00.sum()

- <U>refund*빈도수 데이터가 결제한 데이터의 경우가 미결제 데이터보다 높다.  
- 이로 인해 환불금액이 결제여부와 연관성이 있다고 할 수 있다.</U>

#### 2) year변수분석

- 19년도 까지는 연도가 지나면서 결제 여부가 증가했다. 
- 다만, 19년부터 20년사이에 결제데이터의 수가 감소하고 예상환불액은 오히려 미결제자 측에서 급격히 증가하는 등 추세가 최근에 변하였다.
- 그러므로, year 변수는 지불여부에 연관성이 없다고 볼 수 있다.

In [ ]:
df_y_hp=df.groupby('year')['has_paid'].value_counts()
df_y_hp

In [ ]:
df_y_hp.name='count'
df_y_hp=df_y_hp.reset_index()
df_y_hp1=df_y_hp.query("has_paid==1")
df_y_hp0=df_y_hp.query("has_paid==0")
plt.plot(df_y_hp1.year,df_y_hp1['count'],label='결제데이터')
plt.plot(df_y_hp0.year,df_y_hp0['count'],label='미결제데이터')
plt.legend()

In [ ]:
plt.plot(df.query("has_paid==1").groupby('year')['refund'].sum().index,df.query("has_paid==1").groupby('year')['refund'].sum(),label='결제데이터')
plt.plot(df.query("has_paid==0").groupby('year')['refund'].sum().index,df.query("has_paid==0").groupby('year')['refund'].sum(),label='미결제데이터')
plt.legend()